# Exploring the Raw Titanic Dataset
© Explore Data Science Academy

Part of the journey to making a good regression model is to understand the data that we are modelling. To do this, we will perform some exploratory data analysis on the raw data from the [Titanic Kaggle Challenge](https://www.kaggle.com/c/titanic). The purpose of this challenge is to predict the probability of survival for a given passenger, given their boarding details.

### Honour Code

I **YOUR NAME, YOUR SURNAME**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).  

Non-compliance with the honour code constitutes a material breach of contract.

## Imports

In [2]:
import pandas as pd
import numpy as np

## Data

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Data/regression_sprint/titanic_train_raw.csv')
df_clean = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Data/regression_sprint/titanic_train_clean_raw.csv')

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df_clean.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,B96 B98,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,B96 B98,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,B96 B98,S


## Questions

### Question 1

After briefly looking through the data, you may notice that some entries are missing.

Write a function that determines the number of missing entries for a specified column in the dataset. The function should return a `string` detailing the number of missing values.

_**Function Specifications:**_
* Should take a pandas `DataFrame` and a `column_name` as input and return a `string` as output.
* The `string` should detial the number of missing entries in the column.
* Should be generalised to be able to work on _**ANY**_ dataframe.

In [16]:
### START FUNCTION
def total_missing(df,column_name):
    # your code here
    return f"{column_name} has {df[column_name].isna().sum()} missing values"

### END FUNCTION

In [18]:
total_missing(df,'Survived') == "Survived has 0 missing values"

True

_**Expected Outputs:**_
```python
total_missing(df,'Age') == "Age has 177 missing values"
total_missing(df,'Survived') == "Survived has 0 missing values"
```

### Question 2

It would be a good idea to replace some of our missing data. Missing values can be replaced with the either the _mean_ , the _median_ or the _mode_ (in the case of categorical columns). Write a function that takes in as input a dataframe and a column name, and returns the `mean` for numerical columns and the `mode` for non-numerical columns.

_**Function Specifications:**_
* The function should take two inputs: `(df, column_name)`, where `df` is a pandas `DataFrame`, `column_name` is a `str`.
* If the `column_name` does not exist in `df`, raise a `ValueError`.
* Should return as output the `mean` if the specified column is numerical and return a list of the `mode(s)` otherwise.
* The mean should be rounded to 2 decimal places.
* **If there is more than one `mode` for a given non-numerical column, the fuction should return a list of all modes**. 


In [54]:
### START FUNCTION
def calc_mean_mode(df, column_name):
    # your code here
    try:
        if df[column_name].dtype.kind in 'iufc':
            results = round(df[column_name].mean(), 2)
        else:
            results = list(df[column_name].mode())
    except KeyError:
        raise ValueError(f"Columnd {column_name} does not exist in dataframe df")
        
    return  results

### END FUNCTION

In [55]:
calc_mean_mode(df,'Embared')

ValueError: Columnd Embared does not exist in dataframe df

_**Expected Outputs:**_
```python
calc_mean_mode(df, 'Age') == 29.7
calc_mean_mode(df, 'Embarked') == ['S']
```

### Question 3

We ultimately want to predict the survival chances of the passengers in the testing set. We can start by building a simple model using the data we already have by using _conditional probability_ ! Write a function that returns the survival probability of a passenger, given a condition on a **numerical variable** from the dataset. The condition will consist of a `column_name`, a `value` and a `boolean_operator`. Possible boolean operators include `"<"`,`">"`, or `"=="`. For example, `column_name = "Age"`, `boolean_operator = ">"`, and `value = 40` together form the condition `Age > 40`.

_**Function specifications:**_
* The function should make use of the `df_clean` `DataFrame` loaded earlier in this notebook.
* It should take a numerical `column_name` string, a `boolean_operator` string, and a `value` of type string as input. 
* It should return a survival likelihood as a number between 0 and 1, rounded to 2 decimal places. 
* Assume that `column_name` exists in `df_clean`.

_**Hint:** You can use `eval()` to evaluate string boolean expressions._

In [86]:
### START FUNCTION
def survival_likelihood(df_clean,column_name, boolean_operator, value):
    # your code here
    return round(len(df_clean[column_name][(eval(f'df_clean[\"{column_name}\"]' + boolean_operator + value)) & (df_clean['Survived'] == 1)])/len(df_clean[column_name][eval(f'df_clean[\"{column_name}\"]' + boolean_operator + value)]), 2)

### END FUNCTION

In [88]:
survival_likelihood(df_clean,"Pclass","==","3")

0.24

_**Expected Outputs:**_
```python
survival_likelihood(df,"Pclass","==","3") == 0.24
survival_likelihood(df,"Age","<","15") == 0.58
```